In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from collections import Counter
import re
import datetime as dt
import itertools
print(os.listdir("../input"))

In [2]:
data = pd.read_csv('../input/imbd-sf/data.csv')
#data = pd.read_csv('data.csv') 
data.head(5)

,imdb_id,popularity,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year
0,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015
1,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015
2,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015
3,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015
4,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1890 entries, 0 to 1889
Data columns (total 16 columns):
imdb_id                 1890 non-null object
popularity              1890 non-null float64
budget                  1890 non-null int64
revenue                 1890 non-null int64
original_title          1890 non-null object
cast                    1890 non-null object
director                1890 non-null object
tagline                 1890 non-null object
overview                1890 non-null object
runtime                 1890 non-null int64
genres                  1890 non-null object
production_companies    1890 non-null object
release_date            1890 non-null object
vote_count              1890 non-null int64
vote_average            1890 non-null float64
release_year            1890 non-null int64
dtypes: float64(2), int64(5), object(9)
memory usage: 236.4+ KB


Нормальный датасет, без пропусков. Но много списков "склеенных" в строку, что потребует дополнительной обработки при  анализе.

# Предобработка датасета

In [4]:
answer_ls = [] # создадим список с ответами. сюда будем добавлять ответы по мере прохождения теста
# сюда можем вписать создание новых колонок в датасете

# 1. У какого фильма из списка самый большой бюджет?
Варианты ответов:
1. The Dark Knight Rises (tt1345836)
2. Spider-Man 3 (tt0413300)
3. Avengers: Age of Ultron (tt2395427)
4. The Warrior's Way	(tt1032751)
5. Pirates of the Caribbean: On Stranger Tides (tt1298650)

*Формулировка "из списка" (равно как и  формулировка без уточнения "среди всех фильмов в датасете") допускает двоякое прочтение: кто из списка является "абсолютным чемпионом" всего датасета или кто локально является чемпионом. Первый случай автоматически дает и второй, в обратную сторону совпадения может и не быть. С т.з. собственно pandas разница в получении результата будет только в дополнительном фильтре. Тренировки ради сделаю такой фильтр, а ради очистки совести сверю "локальные и глобальные ответы".**

In [5]:
top_5 = ('1. The Dark Knight Rises (tt1345836)'
'2. Spider-Man 3 (tt0413300)'
'3. Avengers: Age of Ultron (tt2395427)'
'4. The Warrior\'s Way (tt1032751)'
'5. Pirates of the Caribbean: On Stranger Tides (tt1298650)')

#отбор данных по фильтру
def from_top(df=data): return df[df.imdb_id.isin(re.findall(r'tt\d{7}', top_5))]

#поскольку предстоит делать эту операцию не раз,сделаем ее функцией
def pick_one(value, df, worst=False): 
    if worst: return df['original_title'][df[value].idxmin()]
    else: return df['original_title'][df[value].idxmax()]

#чтобы контролировать результат и гарантировать обновление записанного ответа в случае
# критических изменений кода
def add_answer(answer):
    try:
        # можно было бы обойтись и без регулярки, простым find, но с регуляркой удобнее обрабатывать случай с ответом вне списка
        answer_ls.append( int(top_5[re.search(str(answer), top_5).start()-3]))
        print('{}. {}'.format(answer_ls[-1], answer))
    except:
        
        print('"{}": такого варианта нет!'.format(answer))
        
    
    return None

#не очень понятно из условия - смотрим, кто в списке абсолютный чемпион или же, кто чемпион списка
#технически это одно и то же, берем второе, как более частный случайЮ и для очистки совести смотрим первое
add_answer(pick_one('budget', from_top(), worst=False))
print(pick_one('budget', data, worst=False))

4. The Warrior's Way
The Warrior's Way


# 2. Какой из фильмов самый длительный (в минутах)
1. The Lord of the Rings: The Return of the King	(tt0167260)
2. Gods and Generals	(tt0279111)
3. King Kong	(tt0360717)
4. Pearl Harbor	(tt0213149)
5. Alexander	(tt0346491)

In [6]:
top_5 = ('1. The Lord of the Rings: The Return of the King	(tt0167260)'
'2. Gods and Generals	(tt0279111)'
'3. King Kong	(tt0360717)'
'4. Pearl Harbor	(tt0213149)'
'5. Alexander	(tt0346491)')

print(pick_one('runtime', data, worst=False))
add_answer(pick_one('runtime', from_top(), worst=False))


Gods and Generals
2. Gods and Generals


# 3. Какой из фильмов самый короткий (в минутах)
Варианты ответов:

1. Home on the Range	tt0299172
2. The Jungle Book 2	tt0283426
3. Winnie the Pooh	tt1449283
4. Corpse Bride	tt0121164
5. Hoodwinked!	tt0443536

In [7]:
top_5 = ('1. Home on the Range	tt0299172'
'2. The Jungle Book 2	tt0283426'
'3. Winnie the Pooh	tt1449283'
'4. Corpse Bride	tt0121164'
'5. Hoodwinked!	tt0443536')


print(pick_one('runtime', data, worst=True))
add_answer(pick_one('runtime', from_top(), worst=True))

Winnie the Pooh
3. Winnie the Pooh


# 4. Средняя длительность фильма?

Варианты ответов:
1. 115
2. 110
3. 105
4. 120
5. 100


In [8]:
top_5 = ('1. 115 2. 110 3. 105 4. 120 5. 100')

add_answer(round(data['runtime'].mean()))

2. 110


# 5. Средняя длительность фильма по медиане?
Варианты ответов:
1. 106
2. 112
3. 101
4. 120
5. 115




In [9]:
top_5 = ('1. 106 2. 112 3. 101 4. 120 5. 115')
add_answer(round(data['runtime'].median()))

1. 106


# 6. Какой самый прибыльный фильм?
Варианты ответов:
1. The Avengers	tt0848228
2. Minions	tt2293640
3. Star Wars: The Force Awakens	tt2488496
4. Furious 7	tt2820852
5. Avatar	tt0499549

In [10]:
data['profit'] = data['revenue']-data['budget']

top_5 = ('1. The Avengers	tt0848228'
'2. Minions	tt2293640'
'3. Star Wars: The Force Awakens	tt2488496'
'4. Furious 7	tt2820852'
'5. Avatar	tt0499549')

print(pick_one('profit', data, worst=False))
add_answer(pick_one('profit', from_top(), worst=False))

Avatar
5. Avatar


# 7. Какой фильм самый убыточный?
Варианты ответов:
1. Supernova tt0134983
2. The Warrior's Way tt1032751
3. Flushed Away	tt0424095
4. The Adventures of Pluto Nash	tt0180052
5. The Lone Ranger	tt1210819

In [11]:
top_5 = ('1. Supernova tt0134983'
'2. The Warrior\'s Way tt1032751'
'3. Flushed Away	tt0424095'
'4. The Adventures of Pluto Nash	tt0180052'
'5. The Lone Ranger	tt1210819')


#проверка на profit < 0 - это решение в общем виде. Тут-то у нас в датасете хватает убыточных фильмов 
#, а в общем случае убыточного фильма может и не быть. Но в дальнейшем коде опустим этот момент компактности ради.
print(pick_one('profit', data.query('profit<0'), worst=True))
add_answer(pick_one('profit', from_top(), worst=True))

The Warrior's Way
2. The Warrior's Way


# 8. Сколько всего фильмов в прибыли?
Варианты ответов:
1. 1478
2. 1520
3. 1241
4. 1135
5. 1398


In [12]:
top_5 = ('1. 1478 2. 1520 3. 1241 4. 1135 5. 1398')
add_answer(len(data.query('profit >0')))


1. 1478


# 9. Самый прибыльный фильм в 2008 году?
Варианты ответов:
1. Madagascar: Escape 2 Africa	tt0479952
2. Iron Man	tt0371746
3. Kung Fu Panda	tt0441773
4. The Dark Knight	tt0468569
5. Mamma Mia!	tt0795421

In [13]:
top_5 = ('1. Madagascar: Escape 2 Africa	tt0479952'
'2. Iron Man	tt0371746'
'3. Kung Fu Panda	tt0441773'
'4. The Dark Knight	tt0468569'
'5. Mamma Mia!	tt0795421')



print(pick_one('revenue', data.query('release_year == 2008'), worst=False))
add_answer(pick_one('profit', data.query('release_year == 2008'), worst=False))

The Dark Knight
4. The Dark Knight


# 10. Самый убыточный фильм за период с 2012 по 2014 (включительно)?
Варианты ответов:
1. Winter's Tale	tt1837709
2. Stolen	tt1656186
3. Broken City	tt1235522
4. Upside Down	tt1374992
5. The Lone Ranger	tt1210819


In [14]:
top_5 = ('1. Winter\'s Tale	tt1837709'
'2. Stolen	tt1656186'
'3. Broken City	tt1235522'
'4. Upside Down	tt1374992'
'5. The Lone Ranger	tt1210819')

add_answer(pick_one('profit', data.query('(release_year >= 2012) and (release_year <=2014)'), worst=True))

5. The Lone Ranger


# 11. Какого жанра фильмов больше всего?
Варианты ответов:
1. Action
2. Adventure
3. Drama
4. Comedy
5. Thriller

In [15]:
top_5 = ('1. Action 2. Adventure 3. Drama 4. Comedy 5. Thriller')


#можно и так: Counter(data.genres.str.split('|').sum()).most_common(1)[0][0]
add_answer(pd.Series(data.genres.str.split('|').sum()).value_counts().index[0])


3. Drama


# 12. Какого жанра среди прибыльных фильмов больше всего?
Варианты ответов:
1. Drama
2. Comedy
3. Action
4. Thriller
5. Adventure

In [16]:
top_5 = ('1. Drama 2. Comedy 3. Action 4. Thriller 5. Adventure')

genres = data.query('profit > 0')['genres'].str.split('|').sum()
add_answer(pd.Series(genres).value_counts().index[0])


1. Drama


# 13. Кто из режиссеров снял больше всего фильмов?
Варианты ответов:
1. Steven Spielberg
2. Ridley Scott 
3. Steven Soderbergh
4. Christopher Nolan
5. Clint Eastwood

In [17]:
top_5 = ('1. Steven Spielberg'
'2. Ridley Scott '
'3. Steven Soderbergh'
'4. Christopher Nolan'
'5. Clint Eastwood')

answer = Counter(data.director.str.split('|').sum()).most_common()[0][0]
add_answer(answer)

3. Steven Soderbergh


# 14. Кто из режиссеров снял больше всего Прибыльных фильмов?
Варианты ответов:
1. Steven Soderbergh
2. Clint Eastwood
3. Steven Spielberg
4. Ridley Scott
5. Christopher Nolan

In [18]:
top_5 = ('1. Steven Soderbergh'
'2. Clint Eastwood'
'3. Steven Spielberg'
'4. Ridley Scott'
'5. Christopher Nolan')


sub_data = data.query('profit > 0')
answer = Counter(sub_data.director.str.split('|').sum()).most_common()[0][0]
add_answer(answer)

4. Ridley Scott


# 15. Кто из режиссеров принес больше всего прибыли?
Варианты ответов:
1. Steven Spielberg
2. Christopher Nolan
3. David Yates
4. James Cameron
5. Peter Jackson


In [19]:
top_5 = ('1. Steven Spielberg'
'2. Christopher Nolan'
'3. David Yates'
'4. James Cameron'
'5. Peter Jackson')

#функкция "расклейки" значений в ячейке по отдельным строкам 
# задаем: что расклеиваем, что берем ключевым параметром для дальнейшей работы и на каком датасете
def split_glued (value, metric, df=data):
    splitted = pd.DataFrame(df[value].str.split('|').to_list(), index=df[metric]).stack().reset_index()
    splitted.columns = splitted.columns[:-1].to_list()+[value]
    return splitted

#а это для лакончиности записи функция дающая ответ на основе сводной таблицы, созданной на "расклеенных" данных
def split_value(value, metric, df=data, worst=False, agg='sum'):
    splitted = split_glued(value, metric, df)
    if worst: 
        return splitted.pivot_table(index=value, values=metric, aggfunc=agg)[metric].idxmin()
    else: 
        return splitted.pivot_table(index=value, values=metric, aggfunc=agg)[metric].idxmax()


add_answer(split_value('director','profit'))

5. Peter Jackson


# 16. Какой актер принес больше всего прибыли?
Варианты ответов:
1. Emma Watson
2. Johnny Depp
3. Michelle Rodriguez
4. Orlando Bloom
5. Rupert Grint

In [20]:
top_5 = ('1. Emma Watson'
'2. Johnny Depp'
'3. Michelle Rodriguez'
'4. Orlando Bloom'
'5. Rupert Grint')

add_answer(split_value('cast','profit'))

1. Emma Watson


# 17. Какой актер принес меньше всего прибыли в 2012 году?
Варианты ответов:
1. Nicolas Cage
2. Danny Huston
3. Kirsten Dunst
4. Jim Sturgess
5. Sami Gayle

In [21]:
top_5 = ('1. Nicolas Cage'
'2. Danny Huston'
'3. Kirsten Dunst'
'4. Jim Sturgess'
'5. Sami Gayle')

add_answer(split_value('cast','profit', df=data.query('release_year == 2012'), worst=True))


3. Kirsten Dunst


# 18. Какой актер снялся в большем количестве высокобюджетных фильмов? (в фильмах где бюджет выше среднего по данной выборке)
Варианты ответов:
1. Tom Cruise
2. Mark Wahlberg 
3. Matt Damon
4. Angelina Jolie
5. Adam Sandler

In [22]:
top_5 = ('1. Tom Cruise'
'2. Mark Wahlberg'
'3. Matt Damon'
'4. Angelina Jolie'
'5. Adam Sandler')

sub_data = data.query('budget > budget.mean()')
answer = Counter(sub_data.cast.str.split('|').sum()).most_common()[0][0]
add_answer(answer)


3. Matt Damon


# 19. В фильмах какого жанра больше всего снимался Nicolas Cage?  
Варианты ответа:
1. Drama
2. Action
3. Thriller
4. Adventure
5. Crime

In [23]:
top_5 = ('1. Drama'
'2. Action'
'3. Thriller'
'4. Adventure'
'5. Crime')


genres = data[data.cast.str.contains('Nicolas Cage')]['genres'].str.split('|').sum()
answer = pd.Series(genres).value_counts().index[0]
add_answer(answer)

2. Action


# 20. Какая студия сняла больше всего фильмов?
Варианты ответа:
1. Universal Pictures (Universal)
2. Paramount Pictures
3. Columbia Pictures
4. Warner Bros
5. Twentieth Century Fox Film Corporation

In [24]:
# это код исключительно для того, чтобы копипастить с вывода списки студий под "зонтиком" главного бренда
# меняя по очереди "зонтик"
df = pd.Series(data.production_companies.str.split('|').sum())
display(df[df.str.contains('Walt')].unique())


array(['Walt Disney Pictures', 'Walt Disney Studios Motion Pictures',
       'Walt Disney Animation Studios', 'Walt Disney',
       'Walt Disney Feature Animation',
       'Walt Disney Television Animation',
       'Walt Disney Animation Australia', 'Walt Disney Animation Canada'],
      dtype=object)

In [25]:
# создаем правило принадлежности студий одному и тому же "зонтику"

studios_dict = {
    'Universal Pictures':['Universal Pictures',  'Universal Animation Studios','Universal Cartoon Studios' ,
'Universal CGI',  'Universal Pictures International (UPI)',
        'Universal Pictures Corporation','Universal Studios',
       'Universal'  ],
    'Warner Bros': ['Warner Bros.', 'Warner Bros. Animation',
        'Warner Bros. Pictures'#, 'Warner Independent Pictures (WIP)'
       ], 
    'Columbia Pictures' : ['Columbia Pictures', 'Columbia TriStar', 
       'Columbia Pictures Corporation'], 
    'Twentieth Century Fox Film Corporation' : ['Twentieth Century Fox Film Corporation',  
          '20th Century Fox', 'Twentieth Century Fox'], 
    'Paramount Pictures': ['Paramount Pictures', 'Paramount Vantage', 'Paramount Animation',
       'Paramount Classics'],
    'Walt Disney' :['Walt Disney Pictures', 
       'Walt Disney Animation Studios', 'Walt Disney',
       'Walt Disney Feature Animation'] 
     
}

#замена студий на зонтики по правилу
def umbrella(x):
    names = x.split('|')
    for i in range(len(names)):
        for studio in studios_dict:
            if names[i] in studios_dict[studio]:
                names[i]=studio
     
    return names 

#один раз для неоднократного использования создаем датасет, разнесенный по отдельным зонтикам и "незонтичным" студиям
#сначала база
split_data = data.loc[:,['imdb_id', 'production_companies']]
#потом зонтики
split_data['umbrellas'] = split_data.production_companies.apply(umbrella)
# разделяем базоу
split_data = pd.DataFrame(split_data.umbrellas.to_list(), index=split_data.imdb_id).stack().reset_index()
# удаляем строительный мусор и наводим порядок
split_data = split_data.drop(columns=split_data.columns[1]).rename({0:'umbrella'}, axis='columns')
#объединяем с исходным датасетом
split_data = split_data.merge(data, on='imdb_id')
#легкий чек - просто разбиение по компаниям дает такое же количество, ничего не потеряли, ничего лишнего не накрутили
len(split_data), len(split_glued('production_companies', 'imdb_id'))

(6079, 6079)

In [26]:
top_5=('1. Universal Pictures (Universal)'
'2. Paramount Pictures'
'3. Columbia Pictures'
'4. Warner Bros'
'5. Twentieth Century Fox Film Corporation')


answer = Counter(split_data.umbrella).most_common()[0][0]
add_answer(answer)
Counter(split_data.umbrella).most_common(3)

1. Universal Pictures


[('Universal Pictures', 187), ('Warner Bros', 186), ('Columbia Pictures', 166)]

# 21. Какая студия сняла больше всего фильмов в 2015 году?
Варианты ответа:
1. Universal Pictures
2. Paramount Pictures
3. Columbia Pictures
4. Warner Bros
5. Twentieth Century Fox Film Corporation

In [27]:
top_5=('1. Universal Pictures'
'2. Paramount Pictures'
'3. Columbia Pictures'
'4. Warner Bros'
'5. Twentieth Century Fox Film Corporation')

sub_data=split_data.query('release_year == 2015')
answer= sub_data.umbrella.value_counts().index[0]
add_answer(answer)


4. Warner Bros


# 22. Какая студия заработала больше всего денег в жанре комедий за все время?
Варианты ответа:
1. Warner Bros
2. Universal Pictures (Universal)
3. Columbia Pictures
4. Paramount Pictures
5. Walt Disney

In [28]:
top_5=('1. Warner Bros'
'2. Universal Pictures (Universal)'
'3. Columbia Pictures'
'4. Paramount Pictures'
'5. Walt Disney')

sub_data = split_data[split_data.genres.str.contains('Comedy')]
answer=sub_data.pivot_table(index='umbrella', values='profit', aggfunc='sum').profit.idxmax()
add_answer(answer)

2. Universal Pictures


# 23. Какая студия заработала больше всего денег в 2012 году?
Варианты ответа:
1. Universal Pictures (Universal)
2. Warner Bros
3. Columbia Pictures
4. Paramount Pictures
5. Lucasfilm

In [29]:
top_5 = ('1. Universal Pictures'
'2. Warner Bros.'
'3. Columbia Pictures'
'4. Paramount Pictures'
'5. Lucasfilm')

sub_data = split_data.query('release_year == 2012') 
answer=sub_data.pivot_table(index='umbrella', values='profit', aggfunc='sum').profit.idxmax()
add_answer(answer)

3. Columbia Pictures


# 24. Самый убыточный фильм от Paramount Pictures
Варианты ответа:

1. K-19: The Widowmaker tt0267626
2. Next tt0435705
3. Twisted tt0315297
4. The Love Guru tt0811138
5. The Fighter tt0964517

In [30]:
top_5 = ('1. K-19: The Widowmaker tt0267626'
'2. Next tt0435705'
'3. Twisted tt0315297'
'4. The Love Guru tt0811138'
'5. The Fighter tt0964517')

answer = split_data.iloc[split_data.query('umbrella == "Paramount Pictures"').profit.idxmin()]['original_title']
add_answer(answer)

1. K-19: The Widowmaker


# 25. Какой Самый прибыльный год (заработали больше всего)?
Варианты ответа:
1. 2014
2. 2008
3. 2012
4. 2002
5. 2015

In [31]:
top_5 = ('1. 2014 2. 2008 3. 2012 4. 2002 5. 2015')

answer=data.pivot_table(index='release_year', values='profit', aggfunc='sum')['profit'].idxmax()
add_answer(answer)

5. 2015


# 26. Какой Самый прибыльный год для студии Warner Bros?
Варианты ответа:
1. 2014
2. 2008
3. 2012
4. 2010
5. 2015

единственный вариант

In [32]:
top_5=('1. 2014 2. 2008 3. 2012 4. 2010 5. 2015')

sub_data = split_data.query('umbrella == "Warner Bros"')
answer=sub_data.pivot_table(index='release_year', values='profit', aggfunc='sum').profit.idxmax()
add_answer(answer)


1. 2014


# 27. В каком месяце за все годы суммарно вышло больше всего фильмов?
Варианты ответа:
1. Январь
2. Июнь
3. Декабрь
4. Сентябрь
5. Май

In [33]:
#весь этот блок исключительно ради автоматизации вывода результата
#ну и ради тренировки в работе с форматами дат
import locale
data.release_date = pd.to_datetime(data.release_date, format='%m/%d/%Y' )
loc=locale.getlocale()
locale.setlocale(locale.LC_ALL, ('RU','UTF8'))
data['release_month']=data['release_date'].dt.strftime('%B')
locale.setlocale(locale.LC_ALL, loc)


top_5 = ('1. Январь'
'2. Июнь'
'3. Декабрь'
'4. Сентябрь'
'5. Май')

answer=data.pivot_table(index='release_month', values='imdb_id', aggfunc='count')['imdb_id'].idxmax()
        
add_answer(answer)


4. Сентябрь


# 28. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)
Варианты ответа:
1. 345
2. 450
3. 478
4. 523
5. 381

In [34]:
top_5 = ('1. 345 2. 450 3. 478 4. 523 5. 381')

answer = len(data[data.release_date.dt.month.isin((6,7,8))])
add_answer(answer)


2. 450


# 29. Какой режисер выпускает (суммарно по годам) больше всего фильмов зимой?
Варианты ответов:
1. Steven Soderbergh
2. Christopher Nolan
3. Clint Eastwood
4. Ridley Scott
5. Peter Jackson

In [35]:
top_5 = ('1. Steven Soderbergh'
'2. Christopher Nolan'
'3. Clint Eastwood'
'4. Ridley Scott'
'5. Peter Jackson')


sub_data = data[data.release_date.dt.month.isin((12,1,2))]

answer = Counter(sub_data.director.str.split('|').sum()).most_common()[0][0]
add_answer(answer)

5. Peter Jackson


# 30. Какой месяц чаще всего по годам самый прибыльный?
Варианты ответа:
1. Январь
2. Июнь
3. Декабрь
4. Сентябрь
5. Май

In [36]:
top_5 = ('1. Январь 2. Июнь 3. Декабрь 4. Сентябрь 5. Май')

by_yr_mth = data.pivot_table(index='release_year', values='profit', columns='release_month' ,aggfunc='sum')
answer = by_yr_mth.apply(lambda row: row.idxmax(), axis=1).value_counts().index[0]
add_answer(answer)

2. Июнь


# 31. Названия фильмов какой студии в среднем самые длинные по количеству символов?
Варианты ответа:
1. Universal Pictures (Universal)
2. Warner Bros.
3. Jim Henson Company, The
4. Paramount Pictures
5. Four By Two Productions

In [37]:
top_5 = ('1. Universal Pictures (Universal)'
'2. Warner Bros'
'3. Jim Henson Company, The'
'4. Paramount Pictures'
'5. Four By Two Productions')


split_data['title_len'] = split_data['original_title'].str.len()
answer=split_data.pivot_table(index='umbrella', values='title_len', aggfunc='mean').title_len.idxmax()
add_answer(answer)

5. Four By Two Productions


# 32. Названия фильмов какой студии в среднем самые длинные по количеству слов?
Варианты ответа:
1. Universal Pictures (Universal)
2. Warner Bros
3. Jim Henson Company, The
4. Paramount Pictures
5. Four By Two Productions

In [38]:
top_5 = ('1. Universal Pictures (Universal)'
'2. Warner Bros'
'3. Jim Henson Company, The'
'4. Paramount Pictures'
'5. Four By Two Productions')



#интересный вопрос, что считать словом, но поскольку мы можем выполнять задания в любом порядке,
#воспользуемся опытом задания 33, считая словами все, что меж пробелов, т.е. чем больше пробелов, тем больше слов

split_data['title_len'] = split_data['original_title'].str.count(' ')
answer=split_data.pivot_table(index='umbrella', values='title_len', aggfunc='mean').title_len.idxmax()
add_answer(answer)

5. Four By Two Productions


# 33. Сколько разных слов используется в названиях фильмов?(без учета регистра)
Варианты ответа:
1. 6540
2. 1002
3. 2461
4. 28304
5. 3432

In [39]:
top_5=('1. 6540'
'2. 1002'
'3. 2461'
'4. 28304'
'5. 3432')

#начинаем с самой грубой фильтрации
answer=len(set(data['original_title'].str.lower().str.split().sum()))

#и, о чудо, она сразу же дает попадание в один из ответов 
#вероятность случайного соовпадения мизерная, будем считать, что это и есть руководящий принцип
#кроме того, даже самый жесткий фильтр сузить до 1002 не даст, 
#да и разбиением слов через дефис поднять их количество до 3432 не полуится
add_answer(answer)


3. 2461


# 34. Какие фильмы входят в 1 процент лучших по рейтингу?
Варианты ответа:
1. Inside Out, Gone Girl, 12 Years a Slave
2. BloodRayne, The Adventures of Rocky & Bullwinkle
3. The Lord of the Rings: The Return of the King
4. 300, Lucky Number Slevin

In [40]:
#поправлю варианты ответа, чтобы было похоже на то, что на платформе
top_5=('1. Inside Out, Gone Girl, 12 Years a Slave'
'2. BloodRayne, The Adventures of Rocky & Bullwinkle'
'3. The Lord of the Rings: The Return of the King, Upside Down'
'4. 300, Lucky Number Slevin')

movies=re.split('\d\. ', top_5)[1:]


best_percent = data.sort_values(by='vote_average', ascending=False)['original_title'].to_list()[:19]
for movie in movies:
    if all([ x in best_percent for x in movie.split(', ')]):
           add_answer(movie)



1. Inside Out, Gone Girl, 12 Years a Slave


# 35. Какие актеры чаще всего снимаются в одном фильме вместе
Варианты ответа:
1. Johnny Depp & Helena Bonham Carter
2. Hugh Jackman & Ian McKellen
3. Vin Diesel & Paul Walker
4. Adam Sandler & Kevin James
5. Daniel Radcliffe & Rupert Grint

In [41]:
top_5=('1. Johnny Depp & Helena Bonham Carter'
'2. Hugh Jackman & Ian McKellen'
'3. Vin Diesel & Paul Walker'
'4. Adam Sandler & Kevin James'
'5. Daniel Radcliffe & Rupert Grint')

#извлечем все, существующие в составах пары актеров
all_existing_pairs=data['cast'].apply(lambda x: list(itertools.combinations(x.split('|'),2))).sum()

#на случай, если есть также победитель не из списка или получится, что пара сформирована "зеркально"
#хотя конкертно на этих данных сработал бы и простой вариант
for pair, n in Counter(all_existing_pairs).most_common():
    if ' & '.join(pair) in top_5: 
        add_answer(' & '.join(pair))
        break           
    if ' & '.join(pair[::-1]) in top_5:
        add_answer(' & '.join(pair[::-1]))
        break 

5. Daniel Radcliffe & Rupert Grint


# 36. У какого из режиссеров выше вероятность выпустить фильм в прибыли? (5 баллов)
Варианты ответа:
1. Quentin Tarantino
2. Steven Soderbergh
3. Robert Rodriguez
4. Christopher Nolan
5. Clint Eastwood

In [42]:
top_5=('1. Quentin Tarantino'
'2. Steven Soderbergh'
'3. Robert Rodriguez'
'4. Christopher Nolan'
'5. Clint Eastwood')

directors = re.split('\d\. ', top_5)[1:]

splitted = split_glued('director', 'profit')

profitable = splitted.query('profit > 0').pivot_table(index='director', values='profit', aggfunc='count')

total = splitted.pivot_table(index='director', values='profit', aggfunc='count')
                                                                                                                     

rate = profitable['profit'] / total['profit']

answer=rate[rate.index.isin(directors)].idxmax()
add_answer(answer)


4. Christopher Nolan


# Submission

In [43]:
len(answer_ls)

36

In [44]:
pd.DataFrame({'Id':range(1,len(answer_ls)+1), 'Answer':answer_ls}, columns=['Id', 'Answer'])

,Id,Answer
0,1,4
1,2,2
2,3,3
3,4,2
4,5,1
5,6,5
6,7,2
7,8,1
8,9,4
9,10,5
